In [4]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("gpreda/chinese-mnist")

print("Path to dataset files:", path)

/Users/quiet98k/Code/ecs171-final-project/.conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 16.5M/16.5M [00:08<00:00, 1.98MB/s]

Extracting files...


Path to dataset files: /Users/quiet98k/.cache/kagglehub/datasets/gpreda/chinese-mnist/versions/7
